To load the necessary libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library


CLIENT_ID = 'PGYPKJK5TVI4PZ1I1D2RTLUYE5E1KC3SBPWB5NEGNAXVX1B3' # your Foursquare ID
CLIENT_SECRET = 'KFEBOQ5SGVDCDA4223TNL2H00JQA3YDPAHEXSEFD0B1GJK5F' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



df_fullcode=pd.read_csv('List of coordinates_Mod.csv')
df_fullcode.rename(columns={'MRT Stations':'Name'},inplace=True)
#df_fullcode=df_fullcode.iloc[0:20,:]

LIMIT = 2000 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius


Your credentails:
CLIENT_ID: PGYPKJK5TVI4PZ1I1D2RTLUYE5E1KC3SBPWB5NEGNAXVX1B3
CLIENT_SECRET:KFEBOQ5SGVDCDA4223TNL2H00JQA3YDPAHEXSEFD0B1GJK5F


In [2]:
df_fullcode.head()

,Name,Lat,Lon,Abbrv
0,Jurong East,1.333415,103.742119,JUR
1,Bukit Batok,1.349073,103.749664,BBT
2,Bukit Gombak,1.358702,103.751787,BGB
3,Choa Chu Kang,1.385092,103.744322,CCK
4,Yew Tee,1.396986,103.747239,YWT


In [3]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    


Function to get all the necessary data

In [4]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent=browse'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['venues']
        # return only relevant information for each nearby venue
        
        for v in results:
            first=v['name']
            second=v['location']['lat']
            third=v['location']['lng']
            try:
                fourth=v['categories'][0]['name']
            except:
                fourth='None'
            
            if (fourth!='None'):
            #if (True):
                venues_list.append([(name,
                                     lat,
                                     lng,
                                     first,
                                     second,
                                     third,
                                     fourth)])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude','Venue Type']
    
    return(nearby_venues)

In [5]:
#sg_venues = getNearbyVenues(names=df_area['Name'],
#                                   latitudes=df_area['Lat'],
#                                   longitudes=df_area['Lon']
#                                  )

sg_venues = getNearbyVenues(names=df_fullcode['Name'],
                            latitudes=df_fullcode['Lat'],
                            longitudes=df_fullcode['Lon']
                           )





Jurong East
Bukit Batok
Bukit Gombak
Choa Chu Kang
Yew Tee
Kranji
Marsiling
Woodlands
Admiralty
Sembawang
Canberra
Yishun
Khatib
Yio Chu Kang
Ang Mo Kio
Bishan
Braddell
Toa Payoh
Novena
Newton
Orchard
Somerset
Dhoby Ghaut
City Hall
Raffles Place
Marina Bay
Marina South Pier
Pasir Ris
Tampines
Simei
Tanah Merah
Bedok
Kembangan
Eunos
Paya Lebar
Aljunied
Kallang
Lavender
Bugis
Tanjong Pagar
Outram Park
Tiong Bahru
Redhill
Queenstown
Commonwealth
Buona Vista
Dover
Clementi
Chinese Garden
Lakeside
Boon Lay
Pioneer
Joo Koon
Gul Circle
Tuas Crescent
Tuas West Road
Tuas Link
Expo
Changi Airport
HarbourFront
Chinatown
Clarke Quay
Little India
Farrer Park
Boon Keng
Potong Pasir
Woodleigh
Serangoon
Kovan
Hougang
Buangkok
Sengkang
Punggol
Bras Basah
Esplande
Promenade
Nicoll Highway
Stadium
Mountbatten
Dakota
MacPherson
Tai Seng
Bartley
Lorong Chuan
Marymount
Caldecott
Bukit Brown
Botanic Gardens
Farrer Road
Holland Village
One-north
Kent Ridge
Haw Par Villa
Pasir Panjang
Labrador Park
Telok Blang

In [6]:
sg_venues.shape


(18472, 7)

In [7]:
sg_venues.head(20)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Type
0,Jurong East,1.333415,103.742119,Jurong East MRT Interchange (NS1/EW24),1.333503,103.742185,Train Station
1,Jurong East,1.333415,103.742119,Westgate,1.334225,103.742824,Shopping Mall
2,Jurong East,1.333415,103.742119,Jurong East MRT Platform (NS/EW),1.333311,103.742223,Platform
3,Jurong East,1.333415,103.742119,Jem,1.333319,103.743117,Shopping Mall
4,Jurong East,1.333415,103.742119,Monster Burger,1.333487,103.741796,Burger Joint
5,Jurong East,1.333415,103.742119,Jurong East Temporary Bus Interchange,1.333259,103.741415,Bus Station
6,Jurong East,1.333415,103.742119,Kopitiam,1.333278,103.741619,Cafeteria
7,Jurong East,1.333415,103.742119,KOPI,1.333494,103.741682,Food Court
8,Jurong East,1.333415,103.742119,Kodak Express Photo-shop,1.333226,103.742151,Miscellaneous Shop
9,Jurong East,1.333415,103.742119,Jollibee,1.333470,103.741659,Fast Food Restaurant


In [8]:
address = 'Singapore'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('{},{}'.format(latitude,longitude))

C:\Anaconda3\envs\Test_env\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


1.2904753,103.8520359


K M-means analysis

In [9]:
# one hot encoding
sg_onehot = pd.get_dummies(sg_venues[['Venue Type']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sg_onehot['Neighbourhood'] = sg_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [sg_onehot.columns[-1]] + list(sg_onehot.columns[:-1])
sg_onehot = sg_onehot[fixed_columns]

sg_onehot.shape

(18472, 574)

In [10]:
sg_grouped = sg_onehot.groupby('Neighbourhood').mean().reset_index()

num_top_venues = 5

for hood in sg_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = sg_grouped[sg_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Admiralty----
             venue  freq
0       Food Court  0.04
1  Doctor's Office  0.03
2             Café  0.03
3    Train Station  0.03
4             Bank  0.03


----Aljunied----
                           venue  freq
0              College Classroom  0.08
1             Chinese Restaurant  0.08
2               Asian Restaurant  0.04
3                    Coffee Shop  0.03
4  Vegetarian / Vegan Restaurant  0.03


----Ang Mo Kio----
               venue  freq
0        Snack Place  0.05
1     Cosmetics Shop  0.04
2             Bakery  0.04
3  Mobile Phone Shop  0.03
4           Pharmacy  0.03


----Bartley----
                                      venue  freq
0                         College Classroom  0.11
1  Residential Building (Apartment / Condo)  0.11
2                               Bus Station  0.07
3                                    Office  0.04
4                                Playground  0.03


----Bayfront----
               venue  freq
0             Office  0.08
1    

In [11]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = sg_grouped['Neighbourhood']

for ind in np.arange(sg_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sg_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Admiralty,Food Court,Parking,Bakery,Medical Center,Café,Doctor's Office,Residential Building (Apartment / Condo),Asian Restaurant,Bank,Train Station
1,Aljunied,College Classroom,Chinese Restaurant,Asian Restaurant,High School,Building,School,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Office
2,Ang Mo Kio,Snack Place,Cosmetics Shop,Bakery,Mobile Phone Shop,Bank,Pharmacy,Lingerie Store,Food Court,Café,Salon / Barbershop
3,Bartley,College Classroom,Residential Building (Apartment / Condo),Bus Station,Office,Concert Hall,College Auditorium,Playground,College Academic Building,Church,Soccer Field
4,Bayfront,Event Space,Office,Jewelry Store,Convention Center,Casino,Hotel,Cosmetics Shop,Coworking Space,Café,Building


In [12]:
# set number of clusters
kclusters = 5

sg_grouped_clustering = sg_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0,max_iter=1000, n_init=100).fit(sg_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 1, 2, 3, 3, 1, 1, 3])

In [13]:
sg_grouped.head()
sg_clustered=pd.DataFrame(sg_grouped['Neighbourhood'])

temp=pd.DataFrame(kmeans.labels_)
sg_clustered['Grouping']=temp

df_fullcode.rename(columns={'Name':'Neighbourhood'},inplace=True)
sg_clustered.set_index('Neighbourhood',inplace=True)
df_fullcode=df_fullcode.reset_index().set_index('Neighbourhood')
sg_clustered=sg_clustered.join(df_fullcode)


sg_clustered=sg_clustered.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'))
sg_clustered.reset_index(inplace=True)




Plotting of the map using folium

In [14]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sg_clustered['Lat'], sg_clustered['Lon'], sg_clustered['Neighbourhood'], sg_clustered['Grouping']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [15]:
sg_clustered.loc[sg_clustered['Grouping'] == 0, sg_clustered.columns[[1] + list(range(5, sg_clustered.shape[1]))]]

,Grouping,Abbrv,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,0,DVR,College Classroom,College Lab,College Engineering Building,College Academic Building,College Theater,College Auditorium,General College & University,Skate Park,School,College Track
104,0,TKK,College Classroom,College & University,College Quad,College Rec Center,College Auditorium,Residential Building (Apartment / Condo),College Theater,Government Building,College Cafeteria,High School
119,0,YCK,College Classroom,College Lab,Bus Line,Tennis Court,Bus Station,Café,Pool Hall,School,College Cafeteria,Parking


In [16]:
sg_clustered.loc[sg_clustered['Grouping'] == 1, sg_clustered.columns[[1] + list(range(5, sg_clustered.shape[1]))]]

,Grouping,Abbrv,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,1,BLY,College Classroom,Residential Building (Apartment / Condo),Bus Station,Office,Concert Hall,College Auditorium,Playground,College Academic Building,Church,Soccer Field
7,1,BDN,Residential Building (Apartment / Condo),Housing Development,Assisted Living,Bus Station,Building,Parking,Playground,Doctor's Office,Office,Convenience Store
8,1,BDR,Residential Building (Apartment / Condo),Gym,College Lab,Coffee Shop,College Library,Bus Station,Arcade,Housing Development,Metro Station,Playground
15,1,BDL,Residential Building (Apartment / Condo),Salon / Barbershop,Housing Development,Bakery,Chinese Restaurant,Convenience Store,Office,Doctor's Office,Bus Station,Playground
25,1,TBA,Residential Building (Apartment / Condo),Asian Restaurant,Coffee Shop,Chinese Restaurant,Bakery,Housing Development,Food Court,Building,Temple,Pool
26,1,CSW,Military Base,Residential Building (Apartment / Condo),Bus Station,Food Court,College Auditorium,School,Coworking Space,Building,Pool,Office
29,1,CNG,Residential Building (Apartment / Condo),Salon / Barbershop,Chinese Restaurant,Building,Basketball Court,Park,Coffee Shop,Parking,Garden,Cosmetics Shop
35,1,DKT,College Classroom,Residential Building (Apartment / Condo),Comedy Club,Food Court,Chinese Restaurant,Building,Other Great Outdoors,Park,High School,Asian Restaurant
43,1,FRR,Residential Building (Apartment / Condo),Chinese Restaurant,Bus Line,Art Gallery,Gym / Fitness Center,Snack Place,Asian Restaurant,Bus Station,Church,Parking
49,1,HVW,College Classroom,Residential Building (Apartment / Condo),Church,Bus Station,Café,Office,College Lab,Bus Stop,Bus Line,Building


In [17]:
sg_clustered.loc[sg_clustered['Grouping'] == 2, sg_clustered.columns[[1] + list(range(5, sg_clustered.shape[1]))]]

,Grouping,Abbrv,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,2,BFT,Event Space,Office,Jewelry Store,Convention Center,Casino,Hotel,Cosmetics Shop,Coworking Space,Café,Building
10,2,BDM,Office,Building,Café,Automotive Shop,Coffee Shop,Bar,Hostel,Chinese Restaurant,Asian Restaurant,Bakery
16,2,BBS,College Classroom,Office,Café,Japanese Restaurant,Building,Church,Coffee Shop,University,College Library,Restaurant
23,2,BNV,Office,Miscellaneous Shop,Café,Salon / Barbershop,Convenience Store,Building,Bakery,Doctor's Office,Shopping Mall,Mexican Restaurant
24,2,CDT,Flower Shop,Building,Office,Residential Building (Apartment / Condo),Bus Stop,Government Building,Bus Station,College Library,College Classroom,Other Great Outdoors
27,2,CGA,Airport Terminal,Office,Gift Shop,Building,Airport Gate,Café,Airport,Coworking Space,Jewelry Store,Snack Place
34,2,COM,Building,Office,Residential Building (Apartment / Condo),Coffee Shop,Asian Restaurant,Tech Startup,Food Court,Non-Profit,Breakfast Spot,Salon / Barbershop
38,2,DTN,Office,Coffee Shop,Café,Japanese Restaurant,Bank,Chinese Restaurant,Building,Salad Place,Doctor's Office,Food Court
48,2,HPV,Office,Automotive Shop,Residential Building (Apartment / Condo),Bus Station,Building,Asian Restaurant,Bar,Parking,Government Building,Coworking Space
52,2,JLB,Office,Hardware Store,Furniture / Home Store,Automotive Shop,Electronics Store,Chinese Restaurant,Residential Building (Apartment / Condo),Thai Restaurant,Hostel,Hotel


In [18]:
sg_clustered.loc[sg_clustered['Grouping'] == 3, sg_clustered.columns[[1] + list(range(5, sg_clustered.shape[1]))]]

,Grouping,Abbrv,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,ADM,Food Court,Parking,Bakery,Medical Center,Café,Doctor's Office,Residential Building (Apartment / Condo),Asian Restaurant,Bank,Train Station
1,3,ALJ,College Classroom,Chinese Restaurant,Asian Restaurant,High School,Building,School,Vegetarian / Vegan Restaurant,Bakery,Coffee Shop,Office
2,3,AMK,Snack Place,Cosmetics Shop,Bakery,Mobile Phone Shop,Bank,Pharmacy,Lingerie Store,Food Court,Café,Salon / Barbershop
5,3,BTW,Chinese Restaurant,Café,Korean Restaurant,Noodle House,Parking,Asian Restaurant,Coffee Shop,Spa,BBQ Joint,Food Court
6,3,BDK,Noodle House,Dessert Shop,Cosmetics Shop,Chinese Restaurant,Café,Electronics Store,Clothing Store,Snack Place,Coffee Shop,Bakery
9,3,BCL,Café,Hotel,Bar,Pub,Karaoke Bar,Food Court,Office,Korean Restaurant,Dance Studio,Asian Restaurant
11,3,BSH,Food Court,Cosmetics Shop,Bakery,Chinese Restaurant,Coffee Shop,Shoe Store,Pharmacy,Japanese Restaurant,Café,Fast Food Restaurant
12,3,BNK,Chinese Restaurant,Noodle House,Coffee Shop,Asian Restaurant,Residential Building (Apartment / Condo),Convenience Store,Bakery,Food Court,Doctor's Office,Salon / Barbershop
13,3,BNL,Bank,Bakery,Japanese Restaurant,Asian Restaurant,Fast Food Restaurant,Café,Salon / Barbershop,Snack Place,Bubble Tea Shop,Chinese Restaurant
14,3,BTN,Women's Store,Furniture / Home Store,Church,Bakery,Boutique,Kids Store,Office,Café,Doctor's Office,Spa


In [19]:
sg_clustered.loc[sg_clustered['Grouping'] == 4, sg_clustered.columns[[1] + list(range(5, sg_clustered.shape[1]))]]

,Grouping,Abbrv,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,4,TBA,Office,Conference Room,Building,Coworking Space,General Entertainment,Cemetery,Meeting Room,Event Space,Metro Station,Non-Profit
46,4,GCL,Office,Building,Bus Station,Coworking Space,Factory,Professional & Other Places,Bus Line,General Travel,Arcade,Hardware Store
53,4,JKN,Office,Building,Bus Station,Asian Restaurant,Fast Food Restaurant,Bus Line,Coffee Shop,Food Court,Factory,Medical Center
55,4,KKB,Office,Building,College Classroom,College Lab,Residential Building (Apartment / Condo),Food Court,High School,Parking,Coworking Space,Bus Station
63,4,LBD,Office,Building,Government Building,Indian Restaurant,Coworking Space,Bus Station,Café,Parking,Business Service,Bus Line
72,4,MRM,Office,Building,Church,Residential Building (Apartment / Condo),Coworking Space,Playground,Parking,Arts & Crafts Store,Fire Station,Automotive Shop
76,4,NCH,Office,Building,Furniture / Home Store,Coworking Space,Asian Restaurant,Café,Indian Restaurant,Tech Startup,Arts & Crafts Store,Athletics & Sports
86,4,PMN,Office,Coworking Space,Building,Conference Room,Metro Station,Café,Cosmetics Shop,Optical Shop,Jewelry Store,Ice Cream Shop
100,4,TSG,Office,Building,Church,Coworking Space,Coffee Shop,Furniture / Home Store,Café,Conference Room,Tech Startup,Breakfast Spot
111,4,TCR,Office,Building,Factory,Coworking Space,Professional & Other Places,College Quad,General Entertainment,Coffee Shop,Automotive Shop,Train Station
